### Raphael Kaya ; Omar Chaouch

# Algorithmen in der Bioinformatik – Übungsblatt 6
## Abgabe: Freitag, 7.6., 10:30 Uhr

Die Abgabe ist mit bis zu 3 Personen möglich und kann auf Deutsch oder Englisch geschrieben sein. Es muss nur eine Person eines Teams abgeben, in diesem Fall müssen die Namen der Teammitglieder hier in diesem Jupyter Notebook ganz oben genannt werden.

Die praktischen Aufgaben sind mit Python3-Code zu lösen.

Laden Sie das bearbeitete Jupyter Notebook im ILIAS hoch.

**Hinweis:** Sie können eine Bilddatei in ihre Abgabe einbinden, indem Sie test.png durch den entsprechenden Dateinamen ersetzen:

In [ ]:
from IPython.display import Image
Image('test.png')

In [ ]:
# simple test() function to check function returns vs. what the function was supposed to return.
def test(got, expected):
    if got == expected:
        prefix = ' OK '
    else:
        prefix = '  X '
    print('%s got: %s expected: %s' % (prefix, repr(got), repr(expected)))

In [ ]:
# Musterlösung für Needleman-Wunsch-Implementierung
def needleman_wunsch(v, w, match_score, mismatch_score, indel_score):
    m = len(v)
    n = len(w)
    d = [[0 for i in range(n + 1)] for _ in range(m + 1)]
    bt = [[None for i in range(n + 1)] for _ in range(m + 1)]
    
    # Rand initialisieren
    for i in range(1, m + 1):
        d[i][0] = indel_score * i
        bt[i][0] = '↑'
    for j in range(1, n + 1):
        d[0][j] = indel_score * j
        bt[0][j] = '←'
        
    # Rekursion
    for i in range(1, m + 1):
        for j in range(1, n + 1):
            pred = []
            pred.append((d[i - 1][j - 1] + match_score if v[i - 1] == w[j - 1] else mismatch_score, '↖'))
            pred.append((d[i][j - 1] + indel_score, '←'))
            pred.append((d[i - 1][j] + indel_score, '↑'))
            pred.sort(key=lambda x: -x[0]) # sortiere absteigend nach Score
            d[i][j] = pred[0][0]
            bt[i][j] = pred[0][1]
            
    # Backtracking (konstruiere Alignments von hinten nach vorne)
    i = m
    j = n
    al = ["", ""]
    while i > 0 or j > 0:
        if bt[i][j] == '↖':
            al[0] += v[i - 1]
            al[1] += w[j - 1]
            i -= 1
            j -= 1
        elif bt[i][j] == '↑':
            al[0] += v[i - 1]
            al[1] += '-'
            i -= 1
        else: # bt[i][j] == '←':
            al[0] += '-'
            al[1] += w[j - 1]
            j -= 1
    
    # drehe Alignments um
    al[0] = al[0][::-1]
    al[1] = al[1][::-1]
    
    return d[m][n], al

## Aufgabe 6.1 (5/5 Punkte)

Implementieren Sie den Smith-Waterman-Algorithmus für lokales Alignment. Sie können dazu die obige Musterlösung für die Needleman-Wunsch-Implementierung vom letzten Blatt als Vorlage benutzen und diese erweitern, aber auch komplett eigenen Code schreiben.

In [11]:
def smith_waterman(v, w, match_score, mismatch_score, indel_score):

    m = len(v)
    n = len(w)
    d = [[0 for i in range(n + 1)] for _ in range(m + 1)]
    bt = [[None for i in range(n + 1)] for _ in range(m + 1)]
    
    max_value = 0
    max_position = (0, 0)
    
    # Rekursion (Änderung: Beginn erst ab einem Match & keine negative Werte)
    for i in range(1, m + 1):
        for j in range(1, n + 1):
            if v[i - 1] == w[j - 1]:
                match_mismatch = match_score
            else:
                match_mismatch = mismatch_score
            
            score_diagonal = d[i - 1][j - 1] + match_mismatch
            score_up = d[i - 1][j] + indel_score
            score_left = d[i][j - 1] + indel_score
            d[i][j] = max(0, score_diagonal, score_up, score_left)
            
            if d[i][j] == score_diagonal:
                bt[i][j] = '↖'
            elif d[i][j] == score_up:
                bt[i][j] = '↑'
            elif d[i][j] == score_left:
                bt[i][j] = '←'
            else:
                bt[i][j] = None

            # Änderung: Max-Wert suchen
            if d[i][j] > max_value:
                max_value = d[i][j]
                max_position = (i, j)

    # Backtracking (Änderung: Backtracking beginnend vom größten Wert)
    i, j = max_position
    al = ["", ""]
    while d[i][j] > 0:
        if bt[i][j] == '↖':
            al[0] += v[i - 1]
            al[1] += w[j - 1]
            i -= 1
            j -= 1
        elif bt[i][j] == '↑':
            al[0] += v[i - 1]
            al[1] += '-'
            i -= 1
        elif bt[i][j] == '←':
            al[0] += '-'
            al[1] += w[j - 1]
            j -= 1

    al[0] = al[0][::-1]
    al[1] = al[1][::-1]
    
    return max_value, al

# simple test() function to check function returns vs. what the function was supposed to return.
def test(got, expected):
    if got == expected:
        prefix = ' OK '
    else:
        prefix = '  X '
    print('%s got: %s expected: %s' % (prefix, repr(got), repr(expected)))
    
test(smith_waterman('AATTAA', 'AAGGAA', 1, -1, -1)[0], 2)
test(smith_waterman('ATACTGG', 'TGACTGAG', 1, -1, -2)[0], 4)
test(smith_waterman('ACTGG', 'TGACTGAG', 1, -1, -2)[0], 4)
test(smith_waterman('ATACTGG', 'TGACTG', 1, -1, -2)[0], 4)

print(smith_waterman('AATTAA', 'AAGGAAC', 1, -1, -1)[1] in [['AA', 'AA'], ['AATTAA', 'AAGGAA']])

 OK  got: 2 expected: 2
 OK  got: 4 expected: 4
 OK  got: 4 expected: 4
 OK  got: 4 expected: 4
True


## Aufgabe 6.2 (7,5/11 Punkte, 1/3 + 1/2 + 5,5/6)

In der Vorlesung wurde der Needleman-Wunsch-Algorithmus erweitert, sodass auch affine Gap-Kosten modelliert werden können. Dazu wurden neben der DP-Matrix $s$ zusätzlich die beiden Matrizen $\stackrel{\rightarrow}{s}$ und $\stackrel{\downarrow}{s}$ verwendet.

(a) Beschreiben Sie, wie jeweils die erste Zeile und erste Spalte der drei **DP-Matrizen** für den Needleman-Wunsch-Algorithmus mit affinen Gap-Kosten initialisiert werden müssen. Begründen Sie, warum die Initialisierung auf diese Weise notwendig ist. Sie können davon ausgehen, dass die Gap-Open-Kosten $\sigma$ immer mindestens so groß wie die Gap-Extend-Kosten $\varepsilon$ sind.

(b) Beschreiben Sie, wie jeweils die erste Zeile und erste Spalte der drei **Backtracking-Matrizen** für den Needleman-Wunsch-Algorithmus mit affinen Gap-Kosten initialisiert werden müssen. Begründen Sie, warum die Initialisierung auf diese Weise notwendig ist. Sie können davon ausgehen, dass die Gap-Open-Kosten $\sigma$ immer mindestens so groß wie die Gap-Extend-Kosten $\varepsilon$ sind.

--------

### Antwort 

### a)

Die DP-Matrix s wird wie gewohnt mit 0 initialisiert. In der ersten Zeile und der ersten Spalte wird keine Bewertung vorgenommen, daher bleiben diese Zellen bei 0. Dies stellt sicher, dass das Alignment ohne zusätzliche Strafen beginnen kann.

Die Matrix $\stackrel{\rightarrow}{s}$ wir in der ersten Zelle / im ersten Element mit Gap-Open-Kosten $\sigma$ initialisiert. Jedes weitere Element wird dann um die Gap-Extend-Kosten $\varepsilon$ erweitert. Das liegt daran, dass die $\stackrel{\rightarrow}{s}$ Matrix für die Rechtsbewegung, also Lücken nach rechts gedacht ist. Damit müssen die Kosten beim "nach rechts" gehen immer nur um $\varepsilon$ erweitert werden. In der Spalte dagegen, hat jedes Element die Gap-Open-Kosten $\sigma$, da es in dieser Matrix keine Bewegung nach unten gibt, nur nach rechts. Daher müsste immer eine neue Lücke aufgemacht werden, weshalb wir die Gap-Open-Kosten $\sigma$ zur Initialisierung jedes Elements nehmen.

Die Matrix $\stackrel{\downarrow}{s}$ wird wie die $\stackrel{\rightarrow}{s}$ wird genau umgekehrt zur $\stackrel{\rightarrow}{s}$ Matrix intiailisiert. Das erste Element der Spalte wird mit den Gap-Open-Kosten $\sigma$ initialisert. Jedes weitere Element in der Spalte wird um die Gap-Extend-Kosten $\varepsilon$ erweitert. In der Zeile wird jedes Element mit Gap-Open-Kosten $\sigma$ initialisiert, da es in dieser Matrix keine Rechtsbewegung gibt. 

Damit haben wir die Kosten für die Eröffnung einer Lücke und die Erweiterung der Lücke richtig miteinbezogen und können die Scoring Matrixen richtig initialisieren.

**Feedback**
- bei s wird, bis auf s0,0 = 0, die erste Spalte wie bei s down und die erste Zeile wie bei s right initialisiert
- die erste Spalte der waagerechten Gaps und die erste Zeile der senkrechten Gaps muss mit -unendlich initialisiert werden, da von dort aus keine waagerechte/senkrechte Gap starten kann

### b)


Matrix s: Die Zeile besteht nur aus rechtsbewegungen. Daher wird hier der Wert aus der $\stackrel{\rightarrow}{s}$ Matrix genommen, um das Backtracking zu initialisieren. Damit setzt sich ein Element immer aus dem Element in der $\stackrel{\rightarrow}{s}$ Matrix zusammen. Bei den Spalten geschieht genau dasselbe, nur wird hier das Element immer mit dem Wert aus der $\stackrel{\downarrow}{s}$ Zelle genommen. Der Wert in i+1,j ist damit derselbe, wie in der Matrix $\stackrel{\rightarrow}{s}$, an der Stelle i+1,j. Und der Wert an der Stelle i,j+1 in der s Matrix ist damit derselbe, wie in der $\stackrel{\downarrow}{s}$ Matrix an der Stelle, i,j+1. In der Spalte und Zeile gibt es demnach keine Pfeile, da in der s Matrix, Pfeile nur diagonal zeigen können.

Matrix $\stackrel{\downarrow}{s}$: Hier erfolgt das Backtracking eines Elemente immer aus dem Wert des Vorgängers plus der Lücke-Erweiterungs-Strafe zusammen. Demnach zeigen die Pfeile nur nach links. In der Spalte werden die Werte der Elemente immer aus dem Wert des Vorgänger plus den Kosten für die Eröffnung einer Lücke initialisiert. Grund ist, dass hier keine Bewegung nach unten erfolgen kann, weshalb in einer anderen Matrix die Bewegung stattfinden muss und um auf dieser Matrix landen zu können, eine neue Lücke eröffnet werden muss. Hier gibt es auch keinen "Pfeil", der auf den Vorgänger zeigt.

Matrix $\stackrel{\downarrow}{s}$: Die Elemente in der Spalte haben als Backtracking Pfeil immer einen Pfeil nach oben. Demnach setzt sich der Wert eines Elements immer aus dem Wert des Vorgängers plus der Lücke-Erweiterungsstrafe zusammen. In der Zeile werden die Elemente aus dem Wert des Vorgängers plus den Kosten für die Eröffnung einer Lücke zusammengesetzt. Denn hier gibt es keine Pfeile nach links, weshalb die Bewegung in einer anderen Matrix zustande gekommen sein muss und um auf diese Matrix zu gelangen, muss eine neue Lücke eröffnet werden. 


**Feedback**

$b$: In der Hauptmatrix sind alle Felder Gap-Closings, d. h., man kommt immer aus einer der beiden anderen Matrizen. Zur Notation: Doppelpfeile ($⇈$ und $⇇$) geben Gap-Openings an, normale Pfeile ($↑$ und $←$) sind Gap-Extensions, Pfeile mit einem Balken dahinter ($⤒$ und $⇤$) sind Gap-Closings. Die Notation kann je nach Lösung natürlich abweichen, solange klar ist, was sie bedeutet.

- $b_{i, 0} = ⤒ \qquad$  für $i = 1, \ldots, m$
- $b_{0, j} = ⇤ \qquad$  für $j = 1, \ldots, n$

$\stackrel{\downarrow}{b}$ und $\stackrel{\rightarrow}{b}$: Gültige Randfelder sind Gap-Extensions, nur die ersten Felder sind Gap-Openings.

- $\stackrel{\downarrow}{b}_{1, 0} = ⇈$ (Gap-Open nach oben)
- $\stackrel{\rightarrow}{b}_{0, 1} = ⇇$ (Gap-Open nach links)
- $\stackrel{\downarrow}{b}_{i, 0} = ↑ \qquad$ für $i = 2, \ldots, m$
- $\stackrel{\rightarrow}{b}_{0, j} = ← \qquad$ für $j = 2, \ldots, n$

Die übrigen Randfelder $\stackrel{\downarrow}{b}_{0, j}$ und $\stackrel{\rightarrow}{b}_{i, 0}$ müssen nicht initialisiert werden, da sie nicht erreicht werden können.

-----

(c) Implementieren Sie einen Algorithmus für globales Alignment mit affinen Gaps. Sie können dazu die obige Musterlösung für die Needleman-Wunsch-Implementierung vom letzten Blatt als Vorlage benutzen und diese erweitern, aber auch komplett eigenen Code schreiben.

Um Teilpunkte zu bekommen, können Sie das Backtracking weglassen und nur den korrekten Alignment-Score berechnen.

In [90]:
def align_affine(v, w, match_score, mismatch_score, gap_open_score, gap_extend_score):
    m = len(v)
    n = len(w)
    s = [[0 for _ in range(n + 1)] for _ in range(m + 1)]
    d = [[float('-inf') for _ in range(n + 1)] for _ in range(m + 1)]
    r = [[float('-inf') for _ in range(n + 1)] for _ in range(m + 1)]
    bt_s = [[None for _ in range(n + 1)] for _ in range(m + 1)]
    bt_d = [[None for _ in range(n + 1)] for _ in range(m + 1)]
    bt_r = [[None for _ in range(n + 1)] for _ in range(m + 1)]

    # Initialisierung der Gap-Kosten
    for i in range(1, m + 1):
        d[i][0] = gap_open_score + (i - 1) * gap_extend_score
        s[i][0] = d[i][0]
        bt_d[i][0] = '↑'
        bt_s[i][0] = '↑'
        
    for j in range(1, n + 1):
        r[0][j] = gap_open_score + (j - 1) * gap_extend_score
        s[0][j] = r[0][j]
        bt_r[0][j] = '←'
        bt_s[0][j] = '←'
        
    # Rekursion
    for i in range(1, m + 1):
        for j in range(1, n + 1):
            d[i][j] = max(d[i-1][j] + gap_extend_score, s[i-1][j] + gap_open_score)
            bt_d[i][j] = '↑' if d[i][j] == d[i-1][j] + gap_extend_score else '↑' 

            r[i][j] = max(r[i][j-1] + gap_extend_score, s[i][j-1] + gap_open_score)
            bt_r[i][j] = '←' if r[i][j] == r[i][j-1] + gap_extend_score else '←' 

            match_or_mismatch = match_score if v[i-1] == w[j-1] else mismatch_score
            s[i][j] = max(s[i-1][j-1] + match_or_mismatch, d[i][j], r[i][j])
            if s[i][j] == s[i-1][j-1] + match_or_mismatch:
                bt_s[i][j] = '↖'
            elif s[i][j] == d[i][j]:
                bt_s[i][j] = '↑'
            else:
                bt_s[i][j] = '←'
            
    # Backtracking
    i, j = m, n
    al = ["", ""]
    bt = bt_s  
    while i > 0 or j > 0:
        if bt == bt_s:
            if bt_s[i][j] == '↖':
                al[0] += v[i - 1]
                al[1] += w[j - 1]
                i -= 1
                j -= 1
            elif bt_s[i][j] == '↑':
                bt = bt_d
            else:
                bt = bt_r
        elif bt == bt_d:
            al[0] += v[i - 1]
            al[1] += '-'
            i -= 1
            bt = bt_s
        else:  
            al[0] += '-'
            al[1] += w[j - 1]
            j -= 1
            bt = bt_s
    
    # Drehe Alignments um
    al[0] = al[0][::-1]
    al[1] = al[1][::-1]
    
    return s[m][n], al

# simple test() function to check function returns vs. what the function was supposed to return.
def test(got, expected):
    if got == expected:
        prefix = ' OK '
    else:
        prefix = '  X '
    print('%s got: %s expected: %s' % (prefix, repr(got), repr(expected)))

test(align_affine('AAAA', 'A', 3, -3, -6, -1)[0], -5)
test(align_affine('AAAAG', 'AG', 3, -3, -6, -1)[0], -2)
test(align_affine('ATTTTG', 'ATG', 1, -1, -2, -1)[0], -1)

# von den folgenden zwei muss nur einer True sein
test(align_affine('ATTTTG', 'ATG', 1, -1, -2, -1)[1], ['ATTTTG', 'A---TG'])
test(align_affine('ATTTTG', 'ATG', 1, -1, -2, -1)[1], ['ATTTTG', 'AT---G'])

test(align_affine('GGGTTAGGGTC', 'GAAACTAGGG', 1, -1, -2, -1)[0], -2)
test(align_affine('CAAAACGCAAAAC', 'CGC', 1, -1, -5, -1)[0], -13)
test(align_affine('AAAA', 'TTTT', 3, -4, -4, -1), (-14, ['AAAA----', '----TTTT']))
test(align_affine('ATTTTG', 'ATG', 1, -1, -2, 0)[0], 1)
     
# von den folgenden zwei muss nur einer True sein
test(align_affine('ATTTTG', 'ATG', 1, -1, -2, 0)[1], ['ATTTTG', 'A---TG'])
test(align_affine('ATTTTG', 'ATG', 1, -1, -2, 0)[1], ['ATTTTG', 'AT---G'])

test(align_affine('AAAG', 'G', 1, -1, -2, 0)[1], ['AAAG', '---G'])
test(align_affine('CAAA', 'C', 1, -1, -2, 0)[1], ['CAAA', 'C---'])


 OK  got: -5 expected: -5
 OK  got: -2 expected: -2
 OK  got: -1 expected: -1
 OK  got: ['ATTTTG', 'A---TG'] expected: ['ATTTTG', 'A---TG']
  X  got: ['ATTTTG', 'A---TG'] expected: ['ATTTTG', 'AT---G']
 OK  got: -2 expected: -2
 OK  got: -13 expected: -13
  X  got: (-14, ['---AAAA', 'TTTT---']) expected: (-14, ['AAAA----', '----TTTT'])
 OK  got: 1 expected: 1
 OK  got: ['ATTTTG', 'A---TG'] expected: ['ATTTTG', 'A---TG']
  X  got: ['ATTTTG', 'A---TG'] expected: ['ATTTTG', 'AT---G']
 OK  got: ['AAAG', '---G'] expected: ['AAAG', '---G']
 OK  got: ['CAAA', 'C---'] expected: ['CAAA', 'C---']


## Aufgabe 6.3 (3/4 Punkte, 1/1 + 2/3)

In der Vorlesung wurden globale Alignments auf mehr als zwei Sequenzen verallgemeinert. Der Needleman-Wunsch-Algorithmus kann so angepasst werden, dass er für eine beliebige Anzahl an Sequenzen funktioniert, allerdings steigt die Laufzeit dabei für $k$ Sequenzen auf $\mathcal{O}(2^k n^k)$. Eine Alternative besteht darin, immer nur paarweise zu alignieren, d. h. mit zwei Sequenzen zu beginnen, aus deren Alignment einen Konsensus zu bilden, diesen gegen die dritte Sequenz zu alignieren, den Konsensus daraus gegen die vierte Sequenz usw.

Der Konsensus für ein (paarweises) Alignment sei wie folgt definiert:
- Bei Match-Positionen (beide Sequenzen haben ein identisches Zeichen) wird das entsprechende Zeichen genommen.
- Bei Mismatch-Positionen wird das lexikographisch kleinere Zeichen genommen. (Theoretisch ist es unbestimmt, aber wir definieren es für diese Aufgabe so, damit der Konsensus eindeutig ist.)
- Bei Gap-Positionen wird das jeweilige Nicht-Gap-Zeichen genommen. (Positionen mit Gaps in beiden Sequenzen kommen nicht vor.)

(a) Implementieren Sie eine Funktion, die den Konsensus eines paarweisen Alignments ausgibt. Die Eingabe ist eine Liste mit zwei gleich langen Strings, also dem zweiten Rückgabewert der Needleman-Wunsch-Implementierung.

In [99]:
def consensus(alignments):
    if len(alignments) != 2 or len(alignments[0]) != len(alignments[1]):
        raise ValueError("Input must be a list with two equally long strings")

    seq1, seq2 = alignments
    consensus_str = []

    for a, b in zip(seq1, seq2):
        if a == b:
            consensus_str.append(a)
        elif a == '-':
            consensus_str.append(b)
        elif b == '-':
            consensus_str.append(a)
        else:
            consensus_str.append(min(a, b))

    return ''.join(consensus_str)

In [100]:
test(consensus(['ACGT', 'ACGT']), 'ACGT')
test(consensus(['ACGT', 'ACAT']), 'ACAT')
test(consensus(['A--T', 'ATAT']), 'ATAT')
test(consensus(['CA--', '--GT']), 'CAGT')

 OK  got: 'ACGT' expected: 'ACGT'
 OK  got: 'ACAT' expected: 'ACAT'
 OK  got: 'ATAT' expected: 'ATAT'
 OK  got: 'CAGT' expected: 'CAGT'


(b) Für ein Alignment aus drei Sequenzen definieren wir den Score wie folgt:<br><br>
Alignment von $3$ Zeichen:<br>
- $3$ Zeichen gleich: $3$ $\cdot$ Match-Score,<br>
- $2$ Zeichen gleich: Match-Score $+$ $2$ $\cdot$ Mismatch-Score,<br>
- Alle Zeichen verschieden: $3$ $\cdot$ Mismatch-Score,<br>

Alignment von $2$ Zeichen, $1$ Gap:<br>
- Beide Zeichen gleich: Match-Score $+$ Indel-Score,<br>
- Beide Zeichen verschieden: Mismatch-Score $+$ Indel-Score,<br>

Alignment von $1$ Zeichen, $2$ Gaps:<br>
- $2$ $\cdot$ Indel-Score.

Für das Beispiel von oben ergäbe sich folgender Score:
```
 C  A  -  C  -  G
 -  A  -  T  T  G
 C  A  A  C  -  -
------------------
 0  3 -2 -1 -2  0 -> Summe = -2
```

Implementieren Sie eine Funktion, die den korrekten Score für ein Alignment aus drei Sequenzen berechnet.

In [106]:
def score_3_way(alignments, match_score, mismatch_score, indel_score):
    
    score = 0

    for i in range(len(alignments[0])):
        col = [alignments[0][i], alignments[1][i], alignments[2][i]]

        if col[0] == col[1] == col[2] and col[0] != '-':
            score += 3 * match_score

        elif col[0] == col[1] != col[2] or col[0] == col[2] != col[1] or col[1] == col[2] != col[0]:
            if '-' in col:
                score += match_score + indel_score
            else:
                score += match_score + 2 * mismatch_score

        elif col[0] != col[1] != col[2] != col[0]:
            if '-' in col:
                score += mismatch_score + indel_score
            else:
                score += 3 * mismatch_score

        if col.count('-') == 2:
            score += 2 * indel_score

    return score

In [108]:
test(score_3_way(['A', 'A', 'A'], 1, -1, -1), 3)
test(score_3_way(['A', 'A', 'C'], 1, -1, -1), -1)
test(score_3_way(['A', 'C', 'C'], 1, -1, -1), -1)
test(score_3_way(['A', 'C', 'G'], 1, -1, -1), -3)
test(score_3_way(['A', 'A', '-'], 1, -1, -1), 0)
test(score_3_way(['-', 'C', 'A'], 1, -1, -1), -2)
test(score_3_way(['-', 'A', '-'], 1, -1, -1), -2)
test(score_3_way(['CA-C-G', '-A-TTG', 'CAAC--'], 1, -1, -1), -2)

#additional testcases (rotated, other scores)
test(score_3_way(['G', 'G', 'T'], 2, -1, -3), 0)
test(score_3_way(['T', 'T', 'G'], 2, -1, -3), 0)
test(score_3_way(['C', 'G', 'T'], 2, -1, -3), -3)
test(score_3_way(['T', 'G', 'A'], 2, -1, -3), -3)
test(score_3_way(['G', 'G', '-'], 2, -1, -3), -1)
test(score_3_way(['-', 'T', 'C'], 2, -1, -3), -4)
test(score_3_way(['-', 'C', '-'], 2, -1, -3), -6)
test(score_3_way(['--AAA--', '-ACAAA-', 'ACAAGAC'], 2, -1, -3), -11)

 OK  got: 3 expected: 3
 OK  got: -1 expected: -1
 OK  got: -1 expected: -1
 OK  got: -3 expected: -3
 OK  got: 0 expected: 0
 OK  got: -2 expected: -2
 OK  got: -2 expected: -2
 OK  got: -2 expected: -2
 OK  got: 0 expected: 0
 OK  got: 0 expected: 0
 OK  got: -3 expected: -3
 OK  got: -3 expected: -3
 OK  got: -1 expected: -1
 OK  got: -4 expected: -4
  X  got: -7 expected: -6
  X  got: -13 expected: -11
